# Unpack Packages

In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
from MSD_utils import get_data_pups, build_time_array, return_average, avg_all, graph_single_variable
from MSD_utils import SD_all, return_SD, range_and_ticks, choose_y_axis_params, data_prep_for_plotting_pups
from MSD_utils import fill_in_and_split, plot_traj_length_histogram, plot_traj, filter_out_short_traj
from MSD_utils import plot_trajectory_overlay, quality_control

from MSD_utils import diffusion_coefficient_point_derivative, diffusion_coefficient_linear_regression
from MSD_utils import calculate_diffusion_coefficients, diffusion_bar_chart, summary_barcharts
from MSD_utils import calculate_MMSDs, plot_general_histogram, plot_MSD_histogram, plot_all_MSD_histograms
from MSD_utils import fillin2, MSD_iteration, vectorized_MMSD_calcs

In [3]:
folder = "./{genotype}/{pup}/{region}/{channel}/"
path = "./{genotype}/geoM2xy_{sample_name}.csv"
frames = 60
SD_frames = [1, 7, 14, 15]
conversion = (0.5, 1.93, 1)#(0.3, 3.95, 1)
to_frame = 59
dimension = "2D"
time_to_calculate = 1

base_name = "WT"
test_bins = np.linspace(0, 75, 76)

# name = 'RED_KO_PEG_P1_S1_cortex'
cut = 4
totvids = 4
frame_m = 60  # atm I can't go lower than the actual value.
conversion = (0.3, 3.95, 1)

parameters = {}
parameters["channels"] = ["RED"]
parameters["genotypes"] = ["WT"]
parameters["pups"] = ["P1", "P2", "P3"]
parameters["surface functionalities"] = ["PEG"]
parameters["slices"] = ["S1", "S2", "S3"]
parameters["regions"] = ["cortex"]
parameters["replicates"] = [1, 2, 3, 4]
parameters["slice suffixes"] = ['a', 'b', 'c']


channels = parameters["channels"]
genotypes = parameters["genotypes"]
pups = parameters["pups"]
surface_functionalities = parameters["surface functionalities"]
slices = parameters["slices"]
regions = parameters["regions"]
replicates = parameters["replicates"]
suffixes = parameters["slice suffixes"]

In [4]:
geoM1x = {}
geoM1y = {}
geoM2xy = {}
SM1x = {}
SM1y = {}
SM2xy = {}

for channel in channels:
    for genotype in genotypes:
        for surface_functionality in surface_functionalities:
            for region in regions:
                for pup in pups:
                    slice_counter = 0
                    for slic in slices:
                        suffix = suffixes[slice_counter]
                        sample_name = "{}_{}_{}_{}_{}_{}".format(channel, genotype, surface_functionality, region, pup, slic)
                        DIR = folder.format(genotype=genotype, pup=pup, region=region, channel=channel)
                        frames, total1, xs, ys, x, y = MSD_iteration(DIR, sample_name, cut, totvids, conversion, frame_m, suffix)
                        
                        geoM1x[sample_name], geoM1y[sample_name], geoM2xy[sample_name], SM1x[sample_name], SM1y[sample_name],\
                            SM2xy[sample_name] = vectorized_MMSD_calcs(frames, total1, xs, ys, x, y, frame_m)
                        np.savetxt(DIR+'geoM2xy_{}.csv'.format(sample_name), geoM2xy[sample_name], delimiter=',')

                        slice_counter = slice_counter + 1

Total particles after merging datasets and filtering short trajectories: 375
Total particles after merging datasets and filtering short trajectories: 506
Total particles after merging datasets and filtering short trajectories: 285
Total particles after merging datasets and filtering short trajectories: 233
Total particles after merging datasets and filtering short trajectories: 1009
Total particles after merging datasets and filtering short trajectories: 555
Total particles after merging datasets and filtering short trajectories: 4390
Total particles after merging datasets and filtering short trajectories: 4479
Total particles after merging datasets and filtering short trajectories: 5138


C:\Users\koolk\Miniconda3\lib\site-packages\scipy\stats\stats.py:314: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(a)


In [5]:
frames = 60

In [6]:
data, avg_over_slices, avg_over_pups, names_with_replicates, time, time_SD, average_over_slices, average_over_pups,\
    all_SD_over_slices, all_SD_over_pups = data_prep_for_plotting_pups(path, frames, SD_frames, conversion, to_frame, parameters)

In [7]:
plot_all_MSD_histograms(parameters, folder, SM2xy, time, test_bins, 1, set_y_limit=True, y_range=1000, set_x_limit=True, x_range=60)

In [8]:
y_range, ticks_y, dec_y, x_range, ticks_x, dec_x = 20, 2.5, 1, 5, 1, 1

In [9]:
in_name1 = "WT"
in_name2 = "RED"

graph_single_variable(average_over_pups, all_SD_over_pups, time, time_SD, SD_frames, in_name1, in_name2, to_frame=to_frame, y_range=y_range,
                      ticks_y=ticks_y, dec_y=dec_y, x_range=x_range, ticks_x=ticks_x, dec_x=dec_x,
                      modify_labels=True, label_identifier="PEG_", x_manual=True, base_name=base_name)
plt.show()